<a href="https://colab.research.google.com/github/quirogaez/capstone/blob/main/notebooks/04_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **04_EDA – Aplicación de Clusterización y Balanceo a Conjuntos de Testeo y Validación**

## Importación y cargue de imágenes

Importar librerías

In [15]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import joblib
import shutil
from tqdm import tqdm
from sklearn.metrics import silhouette_score, silhouette_samples, davies_bouldin_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Definir rutas

In [3]:
# Rutas de las imágenes segmentadas
img_vali = '/content/drive/MyDrive/Capstone/data/pre_procesamiento/segmentadas_sobel/sob_vali'
img_test = '/content/drive/MyDrive/Capstone/data/pre_procesamiento/segmentadas_sobel/sob_test'

# Ruta de los modelos guardados
modelo_dir = '/content/drive/MyDrive/Capstone/modelos'

### Cargar Modelos Guardados

In [4]:
pca = joblib.load(f'{modelo_dir}/pca_pneumonia.pkl')
kmeans = joblib.load(f'{modelo_dir}/kmeans_pneumonia.pkl')

## Preparar Datasets

In [5]:
# Normalización
normalizar = lambda x, y: (tf.cast(x, tf.float32) / 255.0, y)
AUTOTUNE = tf.data.AUTOTUNE

# Cargar datasets
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    img_vali, batch_size = 32, label_mode = 'categorical'
).map(normalizar).prefetch(AUTOTUNE)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    img_test, batch_size = 32, label_mode = 'categorical'
).map(normalizar).prefetch(AUTOTUNE)

class_names = ['NORMAL', 'PNEUMONIA']
indice_pneumonia = class_names.index('PNEUMONIA')

Found 16 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


Función de Conversión a `Numpy`

In [6]:
def dataset_to_numpy(dataset):
    images_list = []
    labels_list = []
    for batch in dataset:
        images, labels = batch
        images_list.append(images.numpy())
        labels_list.append(labels.numpy())
    images_np = np.concatenate(images_list, axis = 0)
    labels_np = np.concatenate(labels_list, axis = 0)
    return images_np, labels_np

Función para Aplicar Modelo

In [7]:
def aplicar_modelo(X, y, pca, kmeans, indice_pneumonia):
    # Filtrar imágenes con neumonía
    mascara_pneumonia = np.argmax(y, axis = 1) == indice_pneumonia
    X_pneumonia = X[mascara_pneumonia]

    # Flatten
    X_pneumonia_flat = X_pneumonia.reshape(X_pneumonia.shape[0], -1)

    # PCA y KMeans
    X_pneumonia_pca = pca.transform(X_pneumonia_flat)
    clusters = kmeans.predict(X_pneumonia_pca)

    # Asignar etiquetas
    df_pneumonia = pd.DataFrame({'Cluster': clusters})
    df_pneumonia['Label'] = df_pneumonia['Cluster'].map({
        0: 'PNEUMONIA-BACTERIAL',
        1: 'PNEUMONIA-VIRAL'
    })

    nuevas_etiquetas = []
    indice_actual = 0

    for idx in range(len(y)):
        if np.argmax(y[idx]) == indice_pneumonia:
            nuevas_etiquetas.append(df_pneumonia.iloc[indice_actual]['Label'])
            indice_actual += 1
        else:
            nuevas_etiquetas.append('NORMAL')

    return nuevas_etiquetas

## Aplicar clusterización a los conjuntos de datos

Conjunto de validación

In [8]:
# Convertir dataset a numpy
X_val, y_val = dataset_to_numpy(val_ds)

# Aplicar modelo
nuevas_etiquetas_val = aplicar_modelo(X_val, y_val, pca, kmeans, indice_pneumonia)

# Mostrar distribución
print(pd.Series(nuevas_etiquetas_val).value_counts())

NORMAL                 8
PNEUMONIA-BACTERIAL    4
PNEUMONIA-VIRAL        4
Name: count, dtype: int64


Conjunto de testeo

In [9]:
# Convertir dataset a numpy
X_test, y_test = dataset_to_numpy(test_ds)

# Aplicar modelo
nuevas_etiquetas_test = aplicar_modelo(X_test, y_test, pca, kmeans, indice_pneumonia)

# Mostrar distribución
print(pd.Series(nuevas_etiquetas_test).value_counts())

NORMAL                 234
PNEUMONIA-BACTERIAL    199
PNEUMONIA-VIRAL        191
Name: count, dtype: int64


## Guardar imágenes clasificadas

Crear las carpetas

In [10]:
# Crear carpetas para validación y test
base_output_dir = '/content/drive/MyDrive/Capstone/data/multiclass'

val_output_dir = f'{base_output_dir}/val'
test_output_dir = f'{base_output_dir}/test'

for path in [val_output_dir, test_output_dir]:
    os.makedirs(f'{path}/NORMAL', exist_ok = True)
    os.makedirs(f'{path}/PNEUMONIA-BACTERIAL', exist_ok = True)
    os.makedirs(f'{path}/PNEUMONIA-VIRAL', exist_ok = True)

Exportar las Imágenes del Conjunto de Validación

In [12]:
# Archivos fuente
pneumonia_files_val = os.listdir(os.path.join(img_vali, 'PNEUMONIA'))
normal_files_val = os.listdir(os.path.join(img_vali, 'NORMAL'))

indice_actual = 0

for i, label in enumerate(nuevas_etiquetas_val):
    if label == 'NORMAL':
        filename = normal_files_val.pop(0)
        src_path = os.path.join(img_vali, 'NORMAL', filename)
        dst_path = os.path.join(val_output_dir, 'NORMAL', filename)
    else:
        filename = pneumonia_files_val[indice_actual]
        src_path = os.path.join(img_vali, 'PNEUMONIA', filename)

        if label == 'PNEUMONIA-BACTERIAL':
            dst_path = os.path.join(val_output_dir, 'PNEUMONIA-BACTERIAL', filename)
        else:
            dst_path = os.path.join(val_output_dir, 'PNEUMONIA-VIRAL', filename)

        indice_actual += 1

    shutil.copy(src_path, dst_path)

Exportar las Imágenes del Conjunto de Testeo

In [14]:
# Archivos fuente
pneumonia_files_test = os.listdir(os.path.join(img_test, 'PNEUMONIA'))
normal_files_test = os.listdir(os.path.join(img_test, 'NORMAL'))

indice_actual = 0

for i, label in enumerate(nuevas_etiquetas_test):
    if label == 'NORMAL':
        filename = normal_files_test.pop(0)
        src_path = os.path.join(img_test, 'NORMAL', filename)
        dst_path = os.path.join(test_output_dir, 'NORMAL', filename)
    else:
        filename = pneumonia_files_test[indice_actual]
        src_path = os.path.join(img_test, 'PNEUMONIA', filename)

        if label == 'PNEUMONIA-BACTERIAL':
            dst_path = os.path.join(test_output_dir, 'PNEUMONIA-BACTERIAL', filename)
        else:
            dst_path = os.path.join(test_output_dir, 'PNEUMONIA-VIRAL', filename)

        indice_actual += 1

    shutil.copy(src_path, dst_path)